<a href="https://colab.research.google.com/github/marine30190/projet_systeme/blob/main/projetSysteme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### System project by Romain VICENS, Marine DALLO and Viktor ILIEV.

# ***System project : HAI724I***





Lors des séquençages haut débit, beaucoup de données sont générées et pour pouvoir analyser toutes ces données, appelées "BigData", les bio informaticiens doivent créer des outils et des programmes puissants.
Le type SAM signifit sequencis aligment mapping, c'est à dire que ce sont les données de l'alignement des reads séquencés grâce aux nouvelles techniques de séquençage.
Le but de ce projet est de pouvoir trier et analyser un fichier de type .sam. 

### **Script permission :**

Start of the text with the permissions to code in Python:

In [ ]:
# !/usr/bin/python3
#-*- coding : utf-8 -*-

###**Prompt authorisations :**
This part concerns the authors and their contacts, the version of the programme as well as its creation date and the licence concerning the rights of copying and use.

In [ ]:
"""__authors__ = ("Romain VICENS", "Marine DALLO", "Viktor ILIEV")
__contact__ = ("romain.vicens@etu.umontpellier.fr","marine.dallo@etu.umontpellier.fr","viktor.iliev@etu.umontpellier.fr")
__version__ = "0.0.1"
__date__ = "05/12/2022"
__licence__ ="This program is free software: you can redistribute it and/or modify
        it under the terms of the GNU General Public License as published by
        the Free Software Foundation, either version 3 of the License, or
        (at your option) any later version.
        This program is distributed in the hope that it will be useful,
        but WITHOUT ANY WARRANTY; without even the implied warranty of
        MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
        GNU General Public License for more details.
        You should have received a copy of the GNU General Public License
        along with this program. If not, see <https://www.gnu.org/licenses/>."""

###**Import of modules :**
This import will allow you to run options like os, sys.argv, sam ...

In [ ]:
import os, sys, sam

###**Checking the file:**

To be able to launch the program on the good type of file it is important to check the nature and the contents of the file which one wishes to analyze. To do this, we will check that the type of the file is not other than file, that is, that it is not a directory. Next, we will check the extension of the file, in this case the extension must be .sam, and we will display an error message if the file or the extension are not those required to run the program. If the file is not a .sam and is empty we stop the program to save time and memory. Then check that the file has content and we will give it a minimum number of 3 lines to say that it is not empty. The 3 lines are justified by the 2 lines of the file identifier and at least one line of read. If the file has less than 3 lines, we will display an error message to indicate that the file is empty.

In [ ]:
if os.path.isdir(sys.argv[1]): #repertoire ou non 
  print("this is a directory")
  sys.exit(1)
elif sys.argv[1].endswith('.sam'):
  print("this the correct file")
else : 
  print("this is not a .sam file")
  print("please give me a unique .sam file as parameter")
  print("usage : python3", sys.argv[0], "tree/file_name.sam")
  sys.exit(1)
        #veryfing if the file is empty
if os.stat(sys.argv[1]).st_size < 3:
  print("The file is empty")
else : 
  print("I have the data")
 

###**Open and read the .sam file:**

This part of the program will open the file and read its contents in order to extract the lines of identification that begin with @, so that they are no longer available when the data is extracted for the analysis of the file. We will also initialise the variables that will be used for the counters used during the statistical readings.

In [ ]:
f = open(sys.argv[1],'r') #ouvre le fichier mis en argument

lines = f.readlines() #stock toutes les lignes du fichier
readtotal=0 #initialiser la variable qui va compter les read totals à 0
cpt_paireend_30=0 #initialisation de la variable qui va compter les paires end à 0
cpt_single_30=0 #initialisation de la variable qui va compter les singles reads à 0
GRsam="Good_Read_Of_Your_Sam.sam" #initialisation de la variable qui va créer un nouveau fichier de sortie
cpt_match = 0 #initialisation du compteur de match
cigar="outpuTable_cigar.txt" #création d'une variable pour la création d'un fichier de sortie
final_cigar="Final_Cigar_table.txt"
txted=open(final_cigar,'w')

#reserch headers ans stockage in variable for not to keep them for the rest of the programme
for line in lines:
	if line[0].startswith('@'):	
		header=line
		with open(GRsam, 'a') as f:
			f.write(header)
	 		#importation des headers dans le nouveau fichier		

###**Checking the number of columns :**

This part will allow you to check the number of columns. This will make it possible to check that the file, even if it is in the right format, has the right number of columns to be sure to have all the desired information. We have chosen to put the limits on the number of columns between 10 and 15. This is justified by the presence or absence of the last columns. They are optional and not present for all reads. If these criteria are not met, we stop the program.

In [ ]:
	else:
		line=line.split()
		if len(line) < 15 and len(line)>10:
			print("this is not a .sam file")
			print("please give me a unique .sam file as parameter")
			sys.exit(1)

###**Creating the dictionary:**

Before creating the dictionary we will split the rows into columns, with a split(), to facilitate the extraction of the data that are present in the rows. This will allow us to create tables in the dictionary and thus extract the relevant information for analysis more easily. In addition to being able to extract more easily, it will allow us to query each column independently and to work with several sorting conditions. We also set a counter to read each extracted read to give the total number of reads.

In [ ]:
else:
		dico = {'ID':line[0], 'FLAG':line[1], 'RNAME':line[2], 'POS':line[3], 'MAPQ':line[4], 'CIGAR':line[5], 'RNEXT':line[6], 'PNEXT':line[7], 'TLEN':line[8], 'SEQ':line[9], 'QUAL':line[10]} #création du dictionnaire
		readtotal+=1 #incrémentation de chaque read pour le compter tous les reads bon ou pas

###**OLD Sorting the pair ends with the Flags:**

In the first version of the code we had thought of sorting the pair-ends according to the flag defined on the referring site of the SAM https://www.samformat.info/sam-format-flag, file and also sorting according to the quality by keeping only the QC > 30 to be very strict on the quality. But we opted for another solution described in the next block in case the Flags numbers would change according to the version. Here our counter counts the reads of each forward and reverse. We therefore increment by 0.5 to give the number of pairs as the final number.

In [ ]:
			"""if ((dico['FLAG'] == '99' or dico['FLAG'] == '147') and int(dico['MAPQ'])>30): 
				paireend += 0.5
			elif ((dico['FLAG'] == '83' or dico['FLAG'] == '163') and int(dico['MAPQ'])>30):
				paireend +=  0.5
			else:
				pass"""

### **Old sorting the pair-ends with the Flags :**

In this new version, we switch the flag columns to binary to avoid errors when changing versions.
To do this we will convert the Flag dico into 12 digit binary and then divide these 12 digits into a list to make it easier for the program to read the code.
We choose to invert the reading of the binary code in the Flags column because Python reads from left to right, unlike us. This will make it easier to translate the binary code to find out the type of the read. 
We will also define a list of tuples with the meaning of each number to be able to profile the read. Example: if it is a well-mapped pair end.

In [ ]:
"""#convert the dico['FLAG'] into binary
def convert_to_binary(dico):
    #convert the dico['FLAG'] into binary with 12 digits
    binary = bin(int(dico['FLAG'])).lstrip('-0b').zfill(12)
    #split the binary into a list
    binary = list(binary)
    #reverse the list
    binary = binary[::-1]
    #define tuple list with the meaning of each digit
    meaning = ('read paired', 'read mapped in proper pair', 'read unmapped', 'mate unmapped', 'read reverse strand', 'mate reverse strand', 'first in pair', 'second in pair', 'not primary alignment', 'read fails platform/vendor quality checks', 'read is PCR or optical duplicate', 'supplementary alignment')"""

###**Selecting flags for pair-ends reads:**

In this part of the code, we will compare the binary digits in the flag column of each read with the list of corresponding flags. We will isolate the flags common to the pairs using the numbers: 110010100000, 110001010000,110010010000 and 110001100000 . These numbers are the equivalent of the flags that represent the reads in well-mapped end-pairs. We will then enter them into an output file and return them.

In [ ]:
			"""if binary[0] == '1' and binary[1] == '1' and binary[2] == '0' and binary[3] == '0' and binary[4] == '1' and binary[5] == '0' and binary[6] == '1' and binary[7] == '0' and binary[8] == '0' and binary[9] == '0' and binary[10] == '0' and binary[11] == '0' and (int(dico['MAPQ'])>=30):
				cpt_paireend_30+=1
				with open(GRsam, 'a') as f:
					f.write("\t".join(dico.values()))
					f.write("\n")
			elif binary[0] == '1' and binary[1] == '1' and binary[2] == '0' and binary[3] == '0' and binary[4] == '0' and binary[5] == '1' and binary[6] == '0' and binary[7] == '1' and binary[8] == '0' and binary[9] == '0' and binary[10] == '0' and binary[11] == '0' and (int(dico['MAPQ'])>=30):
				cpt_paireend_30+=1
				with open(GRsam, 'a') as f:
					f.write("\t".join(dico.values()))
					f.write("\n")
			elif  binary[0] == '1' and binary[1] == '1' and binary[2] == '0' and binary[3] == '0' and binary[4] == '1' and binary[5] == '0' and binary[6] == '0' and binary[7] == '1' and binary[8] == '0' and binary[9] == '0' and binary[10] == '0' and binary[11] == '0' and (int(dico['MAPQ'])>=30):
				cpt_paireend_30+=1
				with open(GRsam, 'a') as f:
					f.write("\t".join(dico.values()))
					f.write("\n")
			elif binary[0] == '1' and binary[1] == '1' and binary[2] == '0' and binary[3] == '0' and binary[4] == '0' and binary[5] == '1' and binary[6] == '1' and binary[7] == '0' and binary[8] == '0' and binary[9] == '0' and binary[10] == '0' and binary[11] == '0' and (int(dico['MAPQ'])>=30):
		#compare each digit of the binary list with the flag list
				cpt_paireend_30+=1
				with open(GRsam, 'a') as f:
					f.write("\t".join(dico.values()))
					f.write("\n")
			elif binary[2]=='0' and binary[3]=='1' and (int(dico['MAPQ'])>=30):
				cpt_single_30 +=1
				with open(GRsam, 'a') as f:
					f.write("\t".join(dico.values()))
					f.write("\n")
			else :
				pass"""

###**New selecting flags for pair-ends reads:**

Improved version of the selection of flags for well-mapped and well-oriented end pairs and for well-mapped and well-oriented singles reads. The selection is still done with the previous binary codes and for the singles reads we have established by comparing with the flags of the badly mapped singles reads that the third binary digit should be 0 and therefore absent and the fourth present and therefore 1. 
This new version is more compact and allows a gain of line of code and thus of execution speed and memory.

In [ ]:
          #convert the dico['FLAG'] into binary with 12 digits, isolate the results where binary is exclusively equals to : 110010100000, 110001010000,110010010000 and 110001100000
            binary = bin(int(dico['FLAG'])).lstrip('-0b').zfill(12)
            binary = '0b' + binary
            if ((binary == '0b000001010011') or (binary == '0b000010100011') or (binary == '0b000001001011') or (binary == '0b000010010011')and (int(dico['MAPQ'])>=30)):
               cpt_paireend_30+=1
            #crate a sam file in the working diractory
                with open(GRsam, 'a') as f:
                    f.write("\t".join(dico.values()))
                    f.write("\n")  
            elif binary[2]=='0' and binary[3]=='1' and (int(dico['MAPQ'])>=30):
                cpt_single_30 +=1 #incrémentation des singles reads
				        with open(GRsam, 'a') as f:
					        f.write("\t".join(dico.values()))
					        f.write("\n")
            else:
                pass

###**CIGAR Analysis :**

We tryed to make a function called in the program after the reading of the .sam file. Supposément la fonction devait lire un fichier généré dans les selections de flag pour les lectures terminaisons-pairées. Cette fonction devait prendre la valeur avant chaque lettre présente dans le cigar, l'assigner à un dictionnaire par correspondance des lettres du cigar. Malheureusement nous n'avons pas réussi à isoler les nombres mais chaque chiffre. Ce problème rends la lecture du fichier final_cigar impossible.

In [ ]:
cigar="outpuTable_cigar.txt"
#function to create a table with the number behind each cigar letter
def cigar_table():
#open the cigar file and read the lines
	f = open(cigar,'r')
	lines = f.readlines()
	for line in lines:
		#define a dictionary with the cigar and the number of occurence
		dico = {'M':0, 'I':0, 'D':0, 'N':0, 'S':0, 'H':0, 'P':0, '=':0, 'X':0}
		#if there is one of the cigar letter in the line, put the digits behind in the matching key in dico
		if 'M' in line:
			dico['M'] = int(line.split('M')[0])
		if 'I' in line:
			dico['I'] = int(line.split('I')[0])
		if 'D' in line:
			dico['D'] = int(line.split('D')[0])
		if 'N' in line:
			dico['N'] = int(line.split('N')[0])
		if 'S' in line:
			dico['S'] = int(line.split('S')[0])
		if 'H' in line:
			dico['H'] = int(line.split('H')[0])
		if 'P' in line:
			dico['P'] = int(line.split('P')[0])
		if '=' in line:
			dico['='] = int(line.split('=')[0])
		if 'X' in line:
			dico['X'] = int(line.split('X')[0])
		#write values in the dictionary in a txt file
		
		with open(final_cigar, 'a') as f:
			f.write("\t".join(str(x) for x in dico.values()))
			f.write("\n")

Calculs : 

Here we print the result of the counter for each condition.

In [ ]:
print("le nombre de read total dans ce fichier est de : ", readtotal)

print("le nombre de paire end bien mappé et bien orienté est de : ", cpt_paireend_30)

print("le nombre de single read bien mappé est de : ",cpt_single_30)

In [ ]:
#check if the file given in parameter is a .sam file
if sys.argv[1].endswith('.sam'):
    #open the file
    with open(sys.argv[1]) as f:
        #read the file
        lines = f.readlines()
        #veryfing if the file is empty
        if os.stat(sys.argv[1]).st_size == 0:
            print("The file is empty")
            sys.exit(1)
        else:
         #read the file
            for line in f:
                #split the line
                line = line.split()
                #check if the line is a header
                if line[0].startswith('@'):
                    #print the header
                    print(line)
                else:
                    #store all the rows in a dictionary
                    dico = {'QNAME':line[0], 'FLAG':line[1], 'RNAME':line[2], 'POS':line[3], 'MAPQ':line[4], 'CIGAR':line[5], 'RNEXT':line[6], 'PNEXT':line[7], 'TLEN':line[8], 'SEQ':line[9], 'QUAL':line[10]}
                    #count the total number of reads
                    total = total + 1
                    #count the number of reads mapped to the reverse strand
                    if dico['FLAG'] == '16':
                        reverse = reverse + 1
                    #count the number of reads mapped to the forward strand
                    elif dico['FLAG'] == '0':
                        forward = forward + 1
                    #count the number of reads mapped to the reverse strand with a mapping quality equal or higher than 30
                    if dico['FLAG'] == '16' and int(dico['MAPQ']) >= 30:
                        reverse30 = reverse30 + 1
                    #count the number of reads mapped to the forward strand with a mapping quality equal or higher than 30
                    elif dico['FLAG'] == '0' and int(dico['MAPQ']) >= 30:
                        forward30 = forward30 + 1
                        #count the number of reads with proerly paired mates
                    elif dico['FLAG'] == '99' or dico['FLAG'] == '147':
                        proper = proper + 1
                    #count the number of reads with a mapping quality equal or higher than 30
                    elif int(dico['MAPQ']) >= 30:
                        mapq30 = mapq30 + 1
                        #count the number of reads totally aligned
                    else :
                        dico['CIGAR'] == '100M'
                        aligned = aligned + 1
    #print the results
    print("Total number of reads: ", total)
    print("Number of reads mapped to the reverse strand: ", reverse)
    print("Number of reads mapped to the forward strand: ", forward)
#    print("Number of reads mapped to the reverse strand with a mapping quality equal or higher than 30: ", reverse30)
#    print("Number of reads mapped to the forward strand with a mapping quality equal or higher than 30: ", forward30)
    print("Number of reads with proerly paired mates: ", proper)
    print("Number of reads with a mapping quality equal or higher than 30: ", mapq30)
    print("Number of reads totally aligned: ", aligned)
else :
    print("This is not a .sam file")
    print("Please give a unique .sam file as parameter")
    print('Usage: python3', sys.argv[0], 'tree/file_name.sam')
    sys.exit(1)
print("The program is finished")

###**First draft of program:**
A first draft of the program was done in the first week of the project. The selection parameters in the "if" conditions did not match the encoding of the .SAM flag provided, and no output file was generated. This code was therefore discarded. In the execution order it looked for the termination of the argument put in position after the call of the program in a bash terminal (i.e. by typing python3 program_name file_name_to_be_analyzed.termination, the termination was checked. If the file was a .sam we checked if it had content, the header was displayed and then the program was analyzed line by line redistributed in a dictionary corresponding to the columns of a sam file. Finally it displayed the counting results in the terminal.
If the file was not a .sam file the program displayed how to use the program before stopping. 


By now we have learned to make python code object oriented, so we build a program with a main and functions. Here is the final program.

In [ ]:
#!/usr/bin/python3
#-*- coding : utf-8 -*-
 
"""__authors__ = ("Romain VICENS", "Marine DALLO", "Viktor ILIEV")
__contact__ = ("romain.vicens@etu.umontpellier.fr","marine.dallo@etu.umontpellier.fr","viktor.iliev@etu.umontpellier.fr")
__version__ = "0.0.1"
__date__ = "05/12/2022"
__licence__ ="This program is free software: you can redistribute it and/or modify
       it under the terms of the GNU General Public License as published by
       the Free Software Foundation, either version 3 of the License, or
       (at your option) any later version.
       This program is distributed in the hope that it will be useful,
       but WITHOUT ANY WARRANTY; without even the implied warranty of
       MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
       GNU General Public License for more details.
       You should have received a copy of the GNU General Public License
       along with this program. If not, see <https://www.gnu.org/licenses/>."""
 
#import of module :
import os, sys, re, time

Here we have the same header as the begining except for the modules : we import re for the exploration of the cigar and time for a "fun" addon if you fail to give a correct sam file.

The following code is the main for the whole program. We chose to detail the main at first so the reader (you) will know how the program execute and then how each function will be explained.

In [ ]:
### Main ###
def main (argv):
   with open(argv[1],'r') as newFile:
       lines : list[Line] = GetListOfLines(newFile.readlines())
       if isDirectory(argv[1]) == True:
           boom()
           sys.exit()
       elif isExtentionValid(argv[1], "sam") == False:
           boom()
           sys.exit()
       elif isFileValid(lines) == False:
           sys.exit()
       totalReads(lines)
       print("What quality of mapping do you want?" + "\n"+ "Answer with a number in range of 0 - 60 ")
       while True:
           quality = int(input())
           if quality not in range(60):
               print("Your desired quality is not in the range, please enter new value for quality (between 0 and 60) ")
           else:
               break
       checkQuality(lines, quality)
       print("Would you like to save a file with the mapped or unmapped reads?" + "\n" + "Answer with: mapped/unmapped")
       x = input()
       if x == "mapped" :
           print("Do you want all mapped reads?" + "\n" + "Answer with : yes/no ")
           y = input()
           if y == "yes":
               result = mapped(checkQuality(lines, quality))
               print("Would you like the cigar procentage of mapped reads?"+ "\n" + "Answer with : yes/no ")
               if input() == "yes":
                   globalPercentCigars(result)
           elif y == "no":
               print("Do you want mapped reads with correct orientation or wrong orientation?"+ "\n" + "Answer with : correct/wrong/both " )
               z = input()
               if z == "correct":
                   mappedRightOrientation(checkQuality(lines, quality))
               elif z == "wrong":
                   mappedWrongOrientation(checkQuality(lines, quality))
               elif z == "both":
                   mappedWrongOrientation(checkQuality(lines, quality))
                   mappedRightOrientation(checkQuality(lines, quality))
       elif x == "unmapped":
           print("Do you want all unmapped reads?"+ "\n" + "Answer with : yes/no ")
           y = input()
           if y == "yes":
               unmapped(checkQuality(lines, quality))
           elif y == "no":
               print("Do you want unmapped reads where single read of the pair in unmapped or where the pair is unmapped?" + "\n" + "Answer with : single/pair/both ")
               z = input()
               if z == "single":
                   singleUnmapped(checkQuality(lines, quality))
               elif z == "pair":
                   doubleUnmapped(checkQuality(lines, quality))
               elif z == "both":
                   singleUnmapped(checkQuality(lines, quality))
                   doubleUnmapped(checkQuality(lines, quality))
 
main(sys.argv)


At first we call the functionn to open the sam file.
We split the file as lines.
We verify if the file is a sam file, if it has data in, if it has the intended number of columns. boom() is a fancy fonction causing to quit the program.
If all those condition are gathered we ask the user what he wants to do, all his responses will activate/unactivate fonctions.

###**Clearing the screen :**
Cleans the terminal.

In [ ]:

def cls():
   os.system('cls' if os.name=='nt' else 'clear')

###**Saving the header as global variable :**
It will be used in the method where we extract a list of the lines in the file


In [ ]:
global header

###**Defining a class for the columns :**
This function will allow to define a class defining the content of each line.

In [ ]:
 
### Defining a class for the columns ###
#Cet fonction va permettre de définir les colonnes dans les quels les lignes du fichier sam seront découpés en colonne
class Line:
   def __init__(self, originalLine, ID, FLAG, RNAME, POS, MAPQ, CIGAR, RNEXT, PNEXT, TLEN, SEQ, QUAL):
       self.originalLine = originalLine
       self.ID = ID
       self.FLAG = FLAG
       self.RNAME = RNAME
       self.POS = POS
       self.MAPQ = MAPQ
       self.CIGAR = CIGAR
       self.RNEXT = RNEXT
       self.PNEXT = PNEXT
       self.TLEN = TLEN
       self.SEQ = SEQ
       self.QUAL = QUAL


###**Str Represent the instance of the originalLine class :**

In [ ]:

   def str(self):
       return self.originalLine


### **Get list of lines using the class method :**
This part of the program allows to save the 2 header lines that start with "@" in a list and then continue the loop.
Then the current line is splitted into a list where we keep only the 11 firsts columns.
This program returns the list of class line where each row is saved in a cell. 

In [ ]:
### Get list of lines using the class method ###
def GetListOfLines(lines) -> list[Line]:
   global header
   header = ""
   list : list[Line] = []
   for line in lines:
       if line.startswith("@"):
           header += line
           continue
       spls = line.split()
       myCurrentLine = Line(line, spls[0], spls[1], spls[2], spls[3], spls[4], spls[5], spls[6], spls[7], spls[8], spls[9], spls[10])
       list.append(myCurrentLine)
   return list


###**Checking if the sam file is good and creating dictionary if it is :**
This function checks if the file is empty and if it contains between 10 and 15 columns. If the result is negative it means the given file cannot be exploited.The whole program shuts down. Here you can see the first iteration of another function (funny but unnecessary, if we had more time we would remplace  the for boucle by boom() ).

--- Sadly the part where it verifies that it contains between 10 and 15 columns isn't well written and doesn't work properly.

In [ ]:

def isFileValid(sam_lines):
   if len(sam_lines) == 0:
       print("The file is empty")
       print("Please give me a valid .sam file as parameter")
       print("The script will shut down in 5 seconds")
       for i in reversed(range(6)):
           print(i)
           time.sleep(1)
       cls()
       print("BOOM")
       return False
   for line in sam_lines:
       if len(sam_lines) < 15 and len(sam_lines) > 10:
           print("This is not a valid .sam file, it has more than 15 columns or less than 10 columns")
           print("Please give me a valid .sam file as parameter")
           print("The script will shut down in 5 seconds")
           for i in reversed(range(6)):
               print(i)
               time.sleep(1)
           cls()
           print("BOOM")
           return False #replacing sys.exit(1) because sys.exit is stopping the program
   return True


###**Checking file's extention :** ###
This function will check if the extension is a .sam file by splitting the argument where there is a "." then comparing what's after the dot with a variable storing "sam".

In [ ]:

def isExtentionValid(file, extentiontype):
   fileExtention = file.split(".")
   if fileExtention[1] == extentiontype:
       return True
   else:
       return False

###**Convert flag into binary :**
This function will convert the flags column into binary to facilitate the search for different versions.
Here the binary is generated with "0b" at the begining, we strip it while covering the eventuality this is a negative number, and add 0 up to 12 times to get a 12 digits binary.
This function transforms the flag into a list and reverse it to make it match the reading direction of python.

Here we should check if the flag is not out of the range of 12digits (breakpoint) in case the sequencer has a memory saturation or if the sam has been modified.

In [ ]:

def flagBinary(flag):
   bFlag = bin(int(flag)).lstrip('-0b').zfill(12) #split the binary into a list   
   binaryFlag = list(bFlag)
   revbinaryFlag = binaryFlag[::-1] #reverse the list
   return revbinaryFlag


###**Total reads in the sam file :**
In this part we will define all the variables used in the rest of the program for all the counters.
It takes out the header and continue to read all lines. If the read is mapped we add 1 to the counter and then continue to check the subtypes if it's correct or wrong orientation and add counter for them respectively. It does the same for unmapped reads and then prints all the the counters as summary of the whole file that was read.

In [ ]:

def totalReads(sam_lines : list[Line]):
   totalReads = 0
   mapped = 0
   mappedRightOrientaion = 0
   mappedWrongOrientation = 0
   unmapped = 0
   singleUnmapped = 0
   doubleUnmapped = 0

   for currline in sam_lines:
       line = currline.str()
       if line.startswith("@"):
           continue
       col_line = line.split()
       flag = flagBinary(col_line[1])
       totalReads += 1
      
 
       if (int(flag[1]) == 1):     #if true = mapped
           mapped += 1
           intFlag = [int(i) for i in flag]
           if (sum(intFlag) == 4):     #if true = right orientation
               mappedRightOrientaion += 1
           else:   #else wrong orientation
               mappedWrongOrientation += 1
       else:   #else unmapped
           unmapped += 1
           if int(flag[2]) and int(flag[3]) == 1: #if true = double
               doubleUnmapped += 1
           else:   #else single
               singleUnmapped += 1
   print("Total reads : " + str(totalReads) + "\n" + "Mapped reads : " + str(mapped) + " from which " + str(mappedRightOrientaion) + " are right orientation and " + str(mappedWrongOrientation) + " are wrong orientation" + "\n" + "Unmapped reads : " + str(unmapped) + " from which " + str(doubleUnmapped) + " are with both reads are unmapped and " + str(singleUnmapped) + " in which one of the reads is unmapped")
 


###**Analysing unmapped - unmapped in general not single or pair :**
This function transforms the flag from the lines in binary and verifies if it contains the unmapped read condition.If it's true in adds 1 to the counter and writes it in export file.
The function returns result which is a list of all reads that answered the condition.

In [ ]:
### Analysing unmapped - unmapped in general not single or pair ###
#This part is dedicated to the mis-mapped read with the Flag reading. 
def unmapped(sam_line):
   unmapped_count = 0
   result = []
   with open("only_unmapped.sam", "a+") as unmapped_fasta, open("summary_unmapped.txt","w") as summary_file: # "a+" is mode for open - append and "w" - write
       unmapped_fasta.write(header)
       for currline in sam_line:
           line = currline.str()
           if line.startswith("@"): # skip header
               continue
           col_line = line.split()
           flag = flagBinary(col_line[1])
 
           if int(flag[2]) == 1:       #only unmapped
               unmapped_count += 1
               unmapped_fasta.write(str(line))
               result.append(currline)
       summary_file.write("Total unmapped reads: " + str(unmapped_count) + "\n")
   print("The files are saved as only_unmapped_both_reads.sam and summary_both_reads_unmapped.txt")
   return result


###**Sub class of unmapped - where both reads in pair are unmapped :**

This function transforms the flag from the lines in binary and verifies if it contains the unmapped read condition and the pair of reads is unmapped.If it's true in adds 1 to the counter and writes it in export file.
The function returns result which is a list of all reads that answered the condition.

In [ ]:
### Sub class of unmapped - where both reads in pair are unmapped ###
# ditto for badly mapped pair-ends
def doubleUnmapped(sam_line):
   doubleCount = 0
   result = []
   with open("only_unmapped_both_reads.sam", "a+") as both_reads_unmapped_fasta, open("summary_both_reads_unmapped.txt", "w") as summary_file_of_both_reads_unmapped:
       both_reads_unmapped_fasta.write(header)
       for currline in sam_line:
           line = currline.str()
           if line.startswith("@"):
               continue
           col_line = line.split()
           flag = flagBinary(col_line[1])
 
       if (int(flag[2]) == 1 and int(flag[3]) == 1):       #unmapped and mate unmapped
           doubleCount += 1
           both_reads_unmapped_fasta.write(str(line))
           result.append(currline)
       summary_file_of_both_reads_unmapped.write("Total unmapped reads where both reads are unmapped: " + str(doubleCount)+ "\n")
   print("The files are saved as only_unmapped_both_reads.sam and summary_both_reads_unmapped.txt")
   return result


###**Sub class of unmapped - where one of the reads in pair is unmapped :**
This function transforms the flag from the lines in binary and verifies if it contains the unmapped read condition and one read of the pair is unmapped.If it's true in adds 1 to the counter and writes it in export file.
The function returns result which is a list of all reads that answered the condition.

In [ ]:
### Sub class of unmapped - where one of the reads in pair is unmapped ###
#ditto for badly mapped singles reads
def singleUnmapped(sam_line):
   singleCount = 0
   result = []
   with open("only_unmapped_single_read.sam", "a+") as single_read_unmapped_fasta, open("summary_single_read_unmapped.txt", "w") as summary_file_of_single_read_unmapped:
       single_read_unmapped_fasta.write(header)
       for currline in sam_line:
           line = currline.str()
           if line.startswith("@"):
               continue
           col_line = line.split()
           flag = flagBinary(col_line[1])
 
           if (int(flag[2]) == 1) or (int(flag[3]) == 1):         #unmapped or mate unmapped
               singleCount += 1
               single_read_unmapped_fasta.write(str(line))
               result.append(currline)
       summary_file_of_single_read_unmapped.write("Total unmapped reads where both reads are unmapped: " + str(singleCount)+ "\n")
   print("The files are saved as only_unmapped_single_read.sam and summary_single_read_unmapped.txt")
   return result
 


###**Analysing mapped reads :**
This function transforms the flag from the lines in binary and verifies if it contains the mapped read condition.If it's true in adds 1 to the counter and writes it in export file.
The function returns result which is a list of all reads that answered the condition.

In [ ]:
### Analysing mapped ###
#ditto for well-mapped reads
def mapped(sam_line) -> list[Line]:
   mapped_count = 0
   result = []
   with open("only_mapped.sam", "a+") as mapped_fasta, open("summary_mapped.txt","w") as summary_file_of_mapped_reads: # "a+" is mode for open - append and "w" - write
       mapped_fasta.write(header)
       for currline in sam_line:
           line = currline.str()
           col_line = line.split()
           flag = flagBinary(col_line[1])
 
           if int(flag[1]) == 1:       #only unmapped
               mapped_count += 1
               mapped_fasta.write(str(line))
               result.append(currline)
       summary_file_of_mapped_reads.write("Total mapped reads: " + str(mapped_count) + "\n" )
   print("The files are saved as only_mapped.sam and summary_mapped.txt")
   return result
 


### **Analysing mapped reads with correct orientation :**
This function transforms the flag from the lines in binary and verifies if it contains the mapped read condition and it's a correct orientation.If it's true in adds 1 to the counter and writes it in export file.
The function returns result which is a list of all reads that answered the condition.

In [ ]:
### Analysing mapped - right orientation ###
def mappedRightOrientation(sam_line):
   rightOrientation = 0
   result = []
   with open("only_mapped_right_orientation.sam", "a+") as right_orientation_mapped_fasta, open("summary_right_orienatiton_mapped.txt","w") as summary_file_of_right_orientation_mapped_reads: # "a+" is mode for open - append and "w" - write
       right_orientation_mapped_fasta.write(header)
       for currline in sam_line:
           line = currline.str()
           if line.startswith("@"): # skip header
               continue
           col_line = line.split()
           flag = flagBinary(col_line[1])
 
           if int(flag[1]) == 1:           #only mapped
               intFlag = list(map(int, flag)) #map function transforming every parameter in the list(flag) to int
               if (sum(intFlag) == 4):       #only correct orientation mapped
                   rightOrientation += 1
                   right_orientation_mapped_fasta.write(str(line))
                   result.append(currline)
       summary_file_of_right_orientation_mapped_reads.write("Total correct orientation mapped reads: " + str(rightOrientation) + "\n")
   print("The files are saved as only_mapped_right_orientation.sam and summary_right_orientation_mapped.txt")
   return result


### **Analysing mapped reads with wrong orientation :**
This function transforms the flag from the lines in binary and verifies if it contains the mapped read condition with wrong orientation.If it's true in adds 1 to the counter and writes it in export file.
The function returns result which is a list of all reads that answered the condition.

In [ ]:
### Analysing mapped - incorrect orientation ###
def mappedWrongOrientation(sam_line):
   wrongOrientation = 0
   result = []
   with open("only_mapped_wrong_orientation.sam", "a+") as wrong_orientation_mapped_fasta, open("summary_wrong_orienatiton_mapped.txt","w") as summary_file_of_wrong_orientation_mapped_reads: # "a+" is mode for open - append and "w" - write
       wrong_orientation_mapped_fasta.write(header)
       for currline in sam_line:
           line = currline.str()
           if line.startswith("@"): # skip header
               continue
           col_line = line.split()
           flag = flagBinary(col_line[1])
 
           if int(flag[1]) == 1:           #only mapped
               intFlag = list(map(int, flag))
               if (sum(intFlag) == 3) or (sum(intFlag) == 5):       #only incorrect orientation mapped
                   wrongOrientation += 1
                   wrong_orientation_mapped_fasta.write(str(line))
                   result.append(currline)
       summary_file_of_wrong_orientation_mapped_reads.write("Total wrong orientation mapped reads: " + str(wrongOrientation) + "\n")
   print("The files are saved as only_mapped_wrong_orientation.sam and summary_wrong_orientation_mapped.txt")
   return result
 


###**Check the quality of the mapping :**
Given a list of `Line` objects from a SAM file, this function verify the MAPQ observed in the lines. It then verifies if MAPQ is equal or superior to the second parameter from the function, if it is then it adds it to list and returns the list.

--- With -> list[Line] we define that the return of the function is mandatory in type of list from class Line (We used it in some of the other functions but due to lack of time we couldn't specify it's type in all of them). This check could be useful if we don't want the functions the return other type than that whichs precised.

In [ ]:
### Check the quality of mapping ###
def checkQuality(lines : list[Line], mapQuality:int) -> list[Line]:
   mylist : list[Line] = []
   for line in lines:
       if int(line.MAPQ) >= mapQuality:
           mylist.append(line)
   return mylist


### **Analysing the file and getting the distribution of cigars in percentage :**
 Given a list of `Line` objects from a SAM file, this function counts the number
 of each type of cigar mutation observed in the lines. It then calculates the global distribution of these mutations as a percentage and returns the result.

In [ ]:
### Analysing the file and getting the distribution of cigars in percentage ###
def globalPercentCigars(lines):
   # Create a dictionary to store the counts for each type of cigar mutation
#we will create a dictionary to list the matches the indels the skipped regions ...
 
   cigar_counts = {
       "M": 0,  # alignment match
       "I": 0,  # insertion
       "D": 0,  # deletion
       "S": 0,  # skipped region
       "H": 0,  # soft clipping
       "N": 0,  # hard clipping
       "P": 0,  # padding
       "X": 0,  # sequence mismatch
       "=": 0,  # sequence match
   }
 
   # Loop through the lines and count the number of each type of cigar mutation
   for line in lines:
       # Skip the header lines
       if line.originalLine.startswith("@"):
           continue
 
       # Parse the cigar string from the line
       cigar_string = line.CIGAR
 
       # Count the number of each type of cigar mutation in the string
       for c in cigar_string:
           if c in cigar_counts:
               cigar_counts[c] += 1
 
   # Calculate the total number of cigar mutations
   total_count = sum(cigar_counts.values())
 
   # Calculate the global distribution of cigar mutations as a percentage
   global_distribution = {
       k: round(100 * v / total_count, 2) for k, v in cigar_counts.items()
   }
 
   # Write the global distribution to a file
   with open("global_distribution_in_percentage.txt", "w") as f:
       for mutation, percentage in global_distribution.items():
           f.write(f"{mutation}: {percentage}\n")
 
   return global_distribution


### **Comments :** 
Sadly due to lack of time we couldn't comment all our functions properly and reduce the use of repeating arguments and in loops.